In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset, random_split

# 数据预处理
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 加载完整的 MNIST 数据集
full_dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)

# 随机选择 20,000 个样本
subset_indices = torch.randperm(len(full_dataset))[:20000]
subset = Subset(full_dataset, subset_indices)

# 将 20,000 个样本分为 10,000 个测试集和 10,000 个训练+验证集
train_val_set, test_set = random_split(subset, [10000, 10000])

# 将 10,000 个训练+验证集样本分为 5,000 个训练集和 5,000 个验证集
train_set, val_set = random_split(train_val_set, [5000, 5000])

# 打乱训练集中的 2,500 个样本的标签
train_indices = torch.randperm(len(train_set))[:2500]
for idx in train_indices:
    train_set.dataset[idx] = (train_set.dataset[idx][0], torch.randint(0, 10, (1,)))

# 创建数据加载器
trainloader = DataLoader(train_set, batch_size=64, shuffle=True)
valloader = DataLoader(val_set, batch_size=64, shuffle=True)
testloader = DataLoader(test_set, batch_size=64, shuffle=True)


In [2]:
## 定义神经网络
N = 5000
la = torch.rand([N,1],requires_grad=True)

class Inner_model(nn.Module):
    def __init__(self):
        super(Inner_model, self).__init__()  
        self.fc = nn.Linear(28*28, 10)

    def forward(self, x):
        x = x.view(-1, 28*28)
        x = self.fc(x)
        return x

net = Inner_model()

In [ ]:
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)
# la_optimizer = optim.Adam([la], lr=1e-4)

N = 5000
la = torch.rand([N,1],requires_grad=True)

def lower_function(outputs, labels, la):
    crossentropy = nn.CrossEntropyLoss()
    loss = crossentropy(outputs, labels)*la
    return loss


for i, data in enumerate(trainloader, 0):
    inputs, labels = data
    outputs = net(inputs)
    loss = lower_function(outputs, labels, la[i])
    loss.backward()
    optimizer.step()

